In [1]:
# https://docs.databricks.com/ingestion/auto-loader/tutorial.html

 

import csv
import uuid
import random
import time
from pathlib import Path

count = 0
path = "/tmp/generated_raw_csv_data"
Path(path).mkdir(parents=True, exist_ok=True)

while True:

    row_list = [ ["id", "x_axis", "y_axis"],
                [uuid.uuid4(), random.randint(-100, 100), random.randint(-100, 100)],
                [uuid.uuid4(), random.randint(-100, 100), random.randint(-100, 100)],
                [uuid.uuid4(), random.randint(-100, 100), random.randint(-100, 100)]
                ]

    file_location = f'{path}/file_{count}.csv' 

    with open(file_location, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(row_list)
        file.close() 

    count += 1
    dbutils.fs.mv(f'file:{file_location}', f'dbfs:{file_location}')
    time.sleep(30)
    print(f'New CSV file created at dbfs:{file_location}. Contents:') 

    with open(f'/dbfs{file_location}', 'r') as file:
        reader = csv.reader(file, delimiter=' ')
        for row in reader:
            print(', '.join(row))
        file.close()

 

"""

This code defines in your workspace the paths to the raw data and the target Delta table, the path to the table’s schema, and the path to the location where Auto Loader writes checkpoint file information in the Delta Lake transaction log. Checkpoints enable Auto Loader to process only new incoming data and to skip over any existing data that has already been processed.

"""

raw_data_location = "dbfs:/tmp/generated_raw_csv_data"
target_delta_table_location = "dbfs:/tmp/table/coordinates"
schema_location = "dbfs:/tmp/auto_loader/schema"
checkpoint_location = "dbfs:/tmp/auto_loader/checkpoint" 

stream = spark.readStream \
    .format("cloudFiles") \
    .option("cloudFiles.format", "csv") \
    .option("header", "true") \
    .option("cloudFiles.schemaLocation", schema_location) \
    .load(raw_data_location)

display(stream)

stream.writeStream \
    .option("checkpointLocation", checkpoint_location) \
    .start(target_delta_table_location)

 

NameError: name 'dbutils' is not defined